## Final Project Submission

Please fill out:
* Student name: Calvin Omwega
* Student pace: Full time
* Scheduled project review date/time: N/A
* Instructor name: Maryann Mwikali
* Blog post URL:N/A


In [13]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

%matplotlib inline

In [14]:

df = pd.read_csv('tmdb.movies.csv', index_col='Unnamed: 0')

In [9]:
new_df = pd.read_csv('bom.movie_gross.csv')

In [10]:
new_df2 = pd.read_csv('tn.movie_budgets.csv', index_col=0)

In [11]:
conn = sqlite3.connect('im.db')

#work